<a href="https://colab.research.google.com/github/Zuhair0000/tensorflow_bootcamp/blob/main/04_transfer_learning_in_tensorflow_part_1_feature_extraction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!nvidia-smi

Tue Feb 10 14:41:44 2026       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   63C    P8             13W /   70W |       0MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [3]:
import zipfile

!wget https://storage.googleapis.com/ztm_tf_course/food_vision/10_food_classes_10_percent.zip

with zipfile.ZipFile('10_food_classes_10_percent.zip', 'r') as zip_ref:
  zip_ref.extractall('.')
  print("Unzipping complete")

--2026-02-10 14:41:44--  https://storage.googleapis.com/ztm_tf_course/food_vision/10_food_classes_10_percent.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 74.125.24.207, 172.217.194.207, 142.250.4.207, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|74.125.24.207|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 168546183 (161M) [application/zip]
Saving to: ‘10_food_classes_10_percent.zip’

10_food_classes_10_ 100%[===================>] 160.74M  23.3MB/s    in 8.1s    

2026-02-10 14:41:52 (20.0 MB/s) - ‘10_food_classes_10_percent.zip’ saved [168546183/168546183]

Unzipping complete


In [4]:
import os

for dirpath, dirnames, filenames in os.walk("/content/10_food_classes_10_percent"):
  print(f"There are {len(dirnames)} directories and {len(filenames)} images in '{dirpath}'. ")

There are 2 directories and 0 images in '/content/10_food_classes_10_percent'. 
There are 10 directories and 0 images in '/content/10_food_classes_10_percent/train'. 
There are 0 directories and 75 images in '/content/10_food_classes_10_percent/train/ramen'. 
There are 0 directories and 75 images in '/content/10_food_classes_10_percent/train/chicken_curry'. 
There are 0 directories and 75 images in '/content/10_food_classes_10_percent/train/steak'. 
There are 0 directories and 75 images in '/content/10_food_classes_10_percent/train/hamburger'. 
There are 0 directories and 75 images in '/content/10_food_classes_10_percent/train/chicken_wings'. 
There are 0 directories and 75 images in '/content/10_food_classes_10_percent/train/grilled_salmon'. 
There are 0 directories and 75 images in '/content/10_food_classes_10_percent/train/pizza'. 
There are 0 directories and 75 images in '/content/10_food_classes_10_percent/train/ice_cream'. 
There are 0 directories and 75 images in '/content/10_fo

In [5]:
train_dir = '/content/10_food_classes_10_percent/train'
test_dir = '/content/10_food_classes_10_percent/test'

In [6]:
import tensorflow as tf

train_data = tf.keras.utils.image_dataset_from_directory(train_dir,
                                                         image_size=(224, 224),
                                                         batch_size=32,
                                                         seed=42)

test_data = tf.keras.utils.image_dataset_from_directory(test_dir,
                                                         image_size=(224, 224),
                                                         batch_size=32,
                                                         seed=42)

Found 750 files belonging to 10 classes.
Found 2500 files belonging to 10 classes.


# Callbacks

### Tensorboard callback

In [7]:
import datetime

def create_tensorboard_callback(dir_name, experiment_name):
  log_dir = dir_name + '/' + experiment_name + "/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
  tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir)
  print(f"Saving TensorBoard log files to : {log_dir}")
  return tensorboard_callback

In [8]:
train_data

<_PrefetchDataset element_spec=(TensorSpec(shape=(None, 224, 224, 3), dtype=tf.float32, name=None), TensorSpec(shape=(None,), dtype=tf.int32, name=None))>